# **Clustering**

**1. Giriş**
<div>
<img src="https://static.javatpoint.com/tutorial/machine-learning/images/clustering-in-machine-learning.png" width="370"/>
</div>

Elimizde her zaman anlamını (etiket değerlerini), neye karşılık geldiğini bildiğimiz veriler olmayabilir. Kümeleme algoritmaları bu tarz verilerin sınıflandırılmasını amaçlar. Sınıflandırmadan tek farkı neyi sınıflandırdığımızı bilmememizdir. Elimizdeki verilerin anlamı/etiketi olmadığı için kümeleme, gözetimsiz öğrenme (unsupervised learning) türüne girer. Temel amacımız bu anlamsız verilerden birbirine benzeyenleri belirli bir sayıda kümeye bölmektir.




**1.1 Problem Tanımı**

Bu çalışmada, kümelemenin en temel algoritması olan K-Means ve daha kompleks algoritmalar olan bazı hiyerarşik aglomeratif kümeleme tekniklerinin implementasyonu gerçekleştirilip verilen 2 boyutlu bir veri seti üzerinde performansları gözlemlenecektir.

**İçindekiler**

2.   [Veri Ön İşlemesi](#cell-id1)

3.   [K-means Clustering](#cell-id2)

4.   [Single Linkage Clustering](#cell-id3)  

5.  [Complete Linkage Clustering](#cell-id4)  


## Modülleri İçe Aktar

In [ ]:
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.figure_factory as ff
import plotly
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

<a name="cell-id1"></a>
## **2.** Veri Ön İşlenmesi

In [ ]:
data = pd.read_csv("/content/clustering_toy_data.csv")
data

,x,y
0,-0.200303,0.510281
1,-0.326642,-0.922599
2,-0.020885,-1.822650
3,-0.796312,0.618746
4,-1.290430,-1.055020
...,...,...
95,2.510740,-3.331230
96,2.424170,-2.143370
97,2.915710,-2.911060
98,2.414640,-3.687660


In [ ]:
x = data.x.to_numpy()
y = data.y.to_numpy()
points = np.stack((x, y), axis=1)

<a name="cell-id2"></a>
## **3.** K-Means Clustering

K-Means algoritması bir gözetimsiz öğrenme (unsupervised learning) ve kümeleme algoritmasıdır.

**Temel K-Means Algoritma Adımları:**
1. Küme sayısı (k) değerini belirle.
2. Başlangıç orta noktalarını rastgele bir şekilde belirle.
3. İlk kümeleri oluştur verileri orta noktalara en yakın kümelere ata.
4. İterasyonlar ile kümeleri daha iyi hale getir.

<div>
<img src="https://files.realpython.com/media/kmeans-algorithm.a94498a7ecd2.png" width="540"/>
</div>

<div>
<img src="https://miro.medium.com/max/600/1*4mMckJCPtQNUy7hm8gCLkg.gif" width="400"/>
</div>


In [ ]:
def visualize_clusters(clusters, centers, center_size=20):
    """
    Visualize clustered 2-D data.
    Parameters
    ----------
    clusters : ndarray
        Clustered data points.
    centers : ndarray
        Centroids.
    center_size : int
        Size of centers to be visualized.
    """
    fig = go.Figure()
    for index, cluster in enumerate(clusters):
        fig.add_trace(go.Scatter(x=cluster[:, 0], y=cluster[:, 1],
                                mode='markers',
                                name='Cluster-' + str(index)))
    fig.add_trace(go.Scatter(x=centers[:, 0], y=centers[:, 1],
                        mode='markers',
                        name='Centroids',
                        marker=dict(size=[center_size] *  len(centers))))
    fig.update_layout(
        title='Visualization of clustered 2D data',
        title_font_size=18,
        width=600,
        height=500,
        xaxis_title='X',
        yaxis_title='Y')
    fig.show()

def SSE(points):
    """
    Calculates the sum of squared errors 
    for the given ndarray of data points.
    Parameters
    ----------
    points : ndarray
        Data points.
    Returns
    -------
    sse : float
        Sum of squared errors.
    """
    centroid = np.mean(points, 0)
    errors = np.linalg.norm(points-centroid, ord=2, axis=1)

    return np.sum(errors)

In [ ]:
class KMeans:
    """
    K-Means clustering algorithm.
    Parameters
    ----------
    n_clusters : int, default=8
        The number of clusters to form as well as the number of
        centroids to generate.
    max_iter : int, default=300
        Maximum number of iterations of the k-means algorithm for a
        single run.
    epochs : int, default=10
        Number of time the k-means algorithm will be run with different centroid seeds. 
        The final results will be the best output
    tol : float, default=1e-4
        Relative tolerance with regards to Frobenius norm of the difference
        in the cluster centers of two consecutive iterations to declare
        convergence.
    seed : int, seed instance or None, default=None
        Determines random number generation for centroid initialization. Use
        an int to make the randomness deterministic.
    Attributes
    ----------
    cluster_centers_ : ndarray of shape (n_clusters, n_features)
        Coordinates of cluster centers. If the algorithm stops before fully
        converging (see ``tol`` and ``max_iter``), these will not be
        consistent with ``labels_``.
    labels_ : ndarray of shape (n_samples,)
        Labels of each point.
    clusters_ : list of ndarray shape (n_clusters,)
        Clustered points are given.
    """
    def __init__(self, n_clusters=5, max_iter=300, tol=1e-4, 
                 epochs=10, seed=None):

        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.n_init = epochs
        self.seed = seed

    def _check_params(self, X):
        # n_init
        if self.n_init <= 0:
            raise ValueError(
                f"n_init should be > 0, got {self.n_init} instead.")
        self._n_init = self.n_init

        # max_iter
        if self.max_iter <= 0:
            raise ValueError(
                f"max_iter should be > 0, got {self.max_iter} instead.")

        # n_clusters
        if X.shape[0] < self.n_clusters:
            raise ValueError(f"n_samples={X.shape[0]} should be >= "
                             f"n_clusters={self.n_clusters}.")

    def _init_centroids(self, X, seed):
        """
        Compute the initial centroids.
        Parameters
        ----------
        X : {ndarray, sparse matrix} of shape (n_samples, n_features)
            The input samples.
        seed : Seed instance
            Determines random number generation for centroid initialization.
        Returns
        -------
        centers : ndarray of shape (n_clusters, n_features)
        """
        n_samples = X.shape[0]
        n_clusters = self.n_clusters
        seeds = np.random.RandomState(seed=seed).permutation(n_samples)[:n_clusters]
        centers = X[seeds]

        return centers

    def fit(self, X):
        """
        Compute K-means clustering.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Training instances to cluster.
        Returns
        -------
        self
            Fitted model.
        """
        self._check_params(X)

        #Infinite float representation in IEEE 754 standards (positive).
        best_error = np.inf 
        for i in range(self.n_init):
            # initialize centroids
            centroids = self._init_centroids(X, seed=self.seed)
            last_error = np.inf
            for j in range(self.max_iter):
                # cluster assignment
                clusters = [None] * self.n_clusters
                labels = np.asarray([None] * X.shape[0])

                for s_index, sample in enumerate(X):
                    index = np.argmin(np.linalg.norm(centroids-sample, 2, 1)) # euclidean L2-norm
                    labels[s_index] = index
                    if clusters[index] is None:
                        clusters[index] = np.expand_dims(sample, 0) # array([1, 2]) -> array([[1, 2]])    
                    else:
                        clusters[index] = np.vstack((clusters[index], sample))

                # centroid update
                centroids = np.asarray([np.mean(cluster, 0) for cluster in clusters])

                # SSE 
                error = np.sum([SSE(cluster) for cluster in clusters])
                gain = last_error - error

                # check for improvement
                if error < best_error:
                    best_clusters, best_labels, best_error = clusters, labels, error

                # epoch break condition
                if np.isclose(gain, 0, atol=self.tol):
                    break
                last_error = error
            
        self.cluster_centers_ = centroids
        self.labels_ = best_labels
        self.clusters_ = best_clusters

        return self

    def predict(self, X):
        """Predict the closest cluster each sample in X belongs to.
        In the vector quantization literature, `cluster_centers_` is called
        the code book and each value returned by `predict` is the index of
        the closest code in the code book.
        Parameters
        -------
        X : {ndarray, sparse matrix} of shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        labels : ndarray of shape (n_samples,)
            Index of the cluster each sample belongs to.
        """
        if self.cluster_centers_ is None:
            raise AssertionError("Model is not fitted!")

        X = np.asarray(X) #initialize ndarray
        labels = np.asarray([None] * X.shape[0])
        for index, point in enumerate(X):
            label = np.argmin(np.linalg.norm(self.cluster_centers_-point, 2, 1)) 
            labels[index] = label

        return labels

In [ ]:
kmeans = KMeans(n_clusters=3).fit(points)

In [ ]:
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=object)

In [ ]:
kmeans.cluster_centers_

array([[ 3.2414488 ,  2.0616374 ],
       [ 3.0974468 , -2.97335674],
       [-0.49818161, -0.04368989]])

In [ ]:
centers = kmeans.cluster_centers_
clusters = kmeans.clusters_
visualize_clusters(clusters, centers)

In [ ]:
kmeans.predict([[2, 3],
                [1, 2],
                [2, -1]])

array([0, 0, 1], dtype=object)

<a name="cell-id3"></a>
## **4.** Single Linkage Clustering

Hiyerarşik kümeleme algoritmaları, benzer nesneleri küme adı verilen gruplar halinde gruplandırır . 

İki tür hiyerarşik kümeleme algoritması vardır:

<div>
<img src="https://miro.medium.com/max/784/0*E-qictlO_9isi0Dl.png" width="490"/>
</div>

**Aglomeratif (Agglomerative) :** Aşağıdan yukarıya yaklaşım. Birçok küçük kümeyle başlayın ve daha büyük kümeler oluşturmak için bunları birleştirin.

**Bölücü (Divider) :** Yukarıdan aşağıya yaklaşım. Daha küçük kümelere bölmek yerine tek bir kümeyle başlayın.

**Artıları**

* Belirli sayıda küme varsayımı yoktur (K-means algoritmasındaki gibi).
* Anlamlı taksonomilere karşılık gelebilir.

**Eksileri**

* İki kümeyi birleştirmek için bir karar verildikten sonra geri alınamaz.
* Büyük veri kümeleri için çok yavaştır, O(𝑛2 log(𝑛)).

Aglomeratif kümeleyiciler nasıl çalışır ?

<div>
<img src="https://d3i71xaburhd42.cloudfront.net/3d28a83e86dc5ab0334ae014b1218f0ad856f76b/6-Figure1-1.png" width="570"/>
</div>

<div>
<img src="https://miro.medium.com/max/875/1*PPmCcLOkDIEVbv1ZAMF18g.png" width="300"/>
</div>

1. İlk önce her veri noktası bir küme olarak belirlenir.

<div>
<img src="https://miro.medium.com/max/875/1*r-XmRRM1VpdIS81ufbtmCQ.png" width="300"/>
</div>

2. Daha sonra en yakın iki veri noktası bir küme olarak birleştirilir.

<div>
<img src="https://miro.medium.com/max/875/1*RrAR6lueQ1cqXi0aNddSvg.png" width="300"/>
</div>

3. Yalnızca bir küme kalana kadar 2. adım tekrarlanır.

**Dendrogramlar**

Gruplandırma geçmişini görselleştirmek ve optimal küme sayısını bulmak için bir dendrogram kullanılabilir.

* Diğer kümelerin hiçbiriyle kesişmeyen en büyük dikey mesafeyi belirleyin.
* Her iki uçta da yatay bir çizgi çizin.
* Optimal küme sayısı, yatay çizgiden geçen dikey çizgilerin sayısına eşittir.

<div>
<img src="https://miro.medium.com/proxy/1*LBOReupihNEsI6Kot3Q6YQ.png" width="500"/>
</div>

Örneğin, aşağıdaki durum için küme sayısı (k) = 4 olacaktır.

**Bağlantı Kriterleri (Linkages)**

Büyük ölçüde farklı sonuçlar elde etmek için belirli parametrelerde ince ayarlar yapılabilir. Bağlantı kriterleri, kümeler arasındaki mesafenin nasıl hesaplandığını ifade eder.

**Tek Bağlantı (Single Linkage)**

Kümede bulunan iki nokta arasındaki en kısa mesafedir ve bu parametre ile kümeleme işlemi her adımda henüz birbirine ait olmayan en yakın eleman çiftini içeren iki kümenin birleştirilmesine dayanır.

<div>
<img src="https://miro.medium.com/max/3000/1*es5FXhkbEGXzYJbljM3kwA.png" width="270"/>
</div>

${\displaystyle D(X,Y)=\min _{x\in X,y\in Y}d(x,y)}$

In [ ]:
def cosine_distance(x, y, data_is_normalized=False):
    """Compute cosine distance between samples in x and y.
    Cosine distance, or the cosine kernel, computes distance as the
    normalized dot product of x and y:
    K(X, Y) = 1 - (<X, Y> / (||X||*||Y||))
    Parameters
    ----------
    x : ([ndarray]) 
        Input array.
    y : ([ndarray])
        Input array.
    data_is_normalized : boolean
        If the samples are normalized, must be True.
    Returns
    ----------
    distance : float 
        Calculated cosine distance.
    """
    if not data_is_normalized:
        dot_product = sum(a*b for a, b in zip(x, y))
        norm_x = sum(a*a for a in x) ** 0.5
        norm_y = sum(b*b for b in y) ** 0.5
    similarity = dot_product / (norm_x * norm_y)
    similarity = np.maximum(0.0, similarity)

    return 1 - similarity

def euclidean_distance(x, y, axis=None):
    """This function calculates 
    the Euclidean L2-norm distance between 
    two given vectors (x, y).
    Parameters
    ----------
    x : ([ndarray]) 
        Input array.
    y : ([ndarray])
        Input array.
    Returns
    ----------
    distance : float
        Calculated Euclidean distance.
    """
    return np.linalg.norm(x - y, ord=2, axis=axis)

def manhattan_distance(x, y):
    """This function calculates 
    the Manhattan distance between 
    two given vectors (x, y).
    Parameters
    ----------
    x : ([ndarray]) 
        Input array.
    y : ([ndarray])
        Input array.
    Returns
    ----------
    distance : float
        Calculated Manhattan distance.
    """
    distance = sum(abs(a-b) for a, b in zip(x, y))
    
    return distance

In [ ]:
class AgglomerativeClustering:
    """
    Agglomerative Clustering
    Recursively merges the pair of clusters that minimally increases
    a given linkage distance.
    Parameters
    ----------
    n_clusters : int or None, default=2
        The number of clusters to find. It must be ``None`` if
        ``distance_threshold`` is not ``None``.
    proximity : str or callable, default='euclidean'
        Metric used to compute the linkage. Can be "euclidean", "l1", "l2",
        "manhattan" or "cosine".
    linkage : {'complete', 'single'}, default='single'
        Which linkage criterion to use. The linkage criterion determines which
        distance to use between sets of observation. The algorithm will merge
        the pairs of cluster that minimize this criterion.
        - 'single' uses the minimum of the distances between all observations
          of the two sets.
        - 'complete' or 'maximum' linkage uses the maximum distances between
          all observations of the two sets.
    Attributes
    ----------
    clusters_ : list of shape (n_clusters)
        The number of clusters found by the algorithm. 
    labels_ : ndarray of shape (n_samples)
        cluster labels for each point
    """
    def __init__(self, n_clusters=2, proximity="euclidean", linkage='single'):
        self.n_clusters = n_clusters
        self.linkage = linkage
        self.proximity = proximity
        if linkage == 'single':
            self._link = self._single_linkage
        elif linkage == 'complete':
            self._link = self._complete_linkage
        if proximity == 'euclidean':
            self._metric = euclidean_distance
        elif proximity == 'manhattan':
            self._metric = manhattan_distance
        elif proximity == 'cosine':
            self._metric = cosine_distance
        else:
            raise ValueError("Proximity metric must be 'euclidean',"
                             "'manhattan' or 'cosine'.")
        
    def _check_params(self, X):
        if self.n_clusters is not None and self.n_clusters <= 0:
            raise ValueError("n_clusters should be an integer greater than 0."
                             " %s was provided." % str(self.n_clusters))
        
        if X.shape[0] < self.n_clusters:
            raise ValueError(f"n_samples={X.shape[0]} should be >= "
                             f"n_clusters={self.n_clusters}.")

        _TREE_BUILDERS = ['single', 'complete']
        if self.linkage not in _TREE_BUILDERS:
            raise ValueError("Unknown linkage type %s. "
                             "Valid options are %s" % (self.linkage,
                                                       _TREE_BUILDERS))

    def _init_proximity_matrix(self, X):
        """
        Compute the initial proximity matrix.
        Parameters
        ----------
        X : {ndarray, sparse matrix} of shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        proximity_matrix : ndarray of shape (n_features, n_features)
        """
        n_samples = X.shape[0]
        proximity_matrix = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                proximity_matrix[i][j] = self._metric(X[i], X[j])

        return proximity_matrix

    def _find_min(self, parent):
        """
        Finds and returns the indexes to be merged.
        Parameters
        ----------
        parent : list
            Samples indexes.
        Returns
        -------
        index : list (c1, c2)
            Indexes to merge.
        """
        min_val = np.inf
        index = [-1, -1]
        for i in range(len(self.proximity_matrix)):
            for j in range(i+1, len(self.proximity_matrix)):
                if min_val > self.proximity_matrix[i][j] and \
                parent[i] == i and parent[j] == j:
                    min_val = self.proximity_matrix[i][j]
                    index = [i, j]

        return index

    def _single_linkage(self, child, first, second, X):
        """Performs single/min/nearest linkage 
        on the condensed proximity matrix.
        linkage='single' assigns
           d(u,v) = \\min(dist(u[i],v[j]))
        Parameters
        ----------
        child : list
            Merged samples indexes.
        first : int indices
        second : int indices
        X : {ndarray, sparse matrix} of shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        min_val : float
            Minimum distance value.
        """
        min_val = np.inf
        for child1 in child[first]:
            for child2 in child[second]:
                min_val = min(min_val, self._metric(X[child1], X[child2]))
                
        return min_val

    def _complete_linkage(self, child, first, second, X):
        """Performs complete/max/farthest point 
        linkage on a condensed proximity matrix.
        linkage='complete' assigns
            d(u, v) = \\max(dist(u[i],v[j]))
        Parameters
        ----------
        child : list
            Merged samples indexes.
        first : int indices
        second : int indices
        X : {ndarray, sparse matrix} of shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        max_val : float
            Maximum distance value.
        """
        max_val = np.NINF
        for child1 in child[first]:
            for child2 in child[second]:
                max_val = max(max_val, self._metric(X[child1], X[child2]))

        return max_val

    def _update_matrix(self, parent, child, nearest_cluster, X):
        """Updates the proximity matrix.
        Parameters
        ----------
        parent : list
            Samples indexes.
        child : list
            Merged samples indexes.
        nearest_cluster : list
        X : {ndarray, sparse matrix} of shape (n_samples, n_features)
            The input samples.
        """
        for i in range(len(self.proximity_matrix)):
            if parent[i] == i:
                self.proximity_matrix[i][nearest_cluster[0]] \
                = self._link(child, nearest_cluster[0], i, X)
                self.proximity_matrix[nearest_cluster[0]][i] \
                = self.proximity_matrix[i][nearest_cluster[0]]
            
    def fit(self, X):
        """Fit the hierarchical agglomerative clustering 
        from features.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features) or (n_samples, n_samples)
            Training instances to cluster.
        Returns
        -------
        self
            Fitted model.
        """
        self._check_params(X)
        n_samples = X.shape[0]
        n_clusters = n_samples
        self.proximity_matrix = self._init_proximity_matrix(X)

        # initialize parent and childs
        parent = [i for i in range(n_samples)]  
        child = [[i] for i in range(n_samples)]
        
        #find clusters
        while n_clusters > self.n_clusters:
            nearest_cluster = self._find_min(parent) # merge 2 cluster
            parent[nearest_cluster[1]] = nearest_cluster[0]
            for c in child[nearest_cluster[1]]:
                child[nearest_cluster[0]].append(c)
            self._update_matrix(parent, child, nearest_cluster, X)
            n_clusters -= 1
        
        clusters_indexes = \
        [child[i] for i in range(n_samples) if parent[i] == i]
        
        clusters = [None] * self.n_clusters
        labels = np.asarray([None] * n_samples)
        for index, point_index in enumerate(clusters_indexes):
            for p_index in point_index:
                labels[p_index] = index
                if clusters[index] is None:
                    clusters[index] = np.expand_dims(X[p_index], 0)    
                else:
                    clusters[index] = np.vstack((clusters[index], X[p_index])) 
                               
        centroids = np.asarray([np.mean(cluster, 0) for cluster in clusters])

        self.cluster_centers_ = centroids
        self.labels_ = labels
        self.clusters_ = clusters
        
        return self

    def predict(self, X):
        """Predict the closest cluster each sample in X belongs to.
        In the vector quantization literature, `cluster_centers_` is called
        the code book and each value returned by `predict` is the index of
        the closest code in the code book.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features) or \
                (n_samples, n_samples)
        Returns
        -------
        labels : ndarray of shape (n_samples,)
            Cluster labels.
        """
        if self.cluster_centers_ is None:
            raise AssertionError("Model is not fitted!")

        X = np.asarray(X) #initialize ndarray
        labels = np.asarray([None] * X.shape[0])
        for index, point in enumerate(X):
            label = np.argmin(self._metric(self.cluster_centers_, point, axis=1))
            labels[index] = label

        return labels

In [ ]:
single = AgglomerativeClustering(linkage='single', n_clusters=3).fit(points)

In [ ]:
single.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=object)

In [ ]:
centers = single.cluster_centers_
clusters = single.clusters_
visualize_clusters(clusters, centers)

In [ ]:
single.predict([[2, 3],
                [1, 2],
                [2, -1]])

array([0, 0, 0], dtype=object)

In [ ]:
single_link = linkage(points, 'single')

fig = ff.create_dendrogram(single_link)
fig.update_layout(width=1200, 
                  height=500,
                  title='Hierarchical Clustering Dendrogram',
                  title_font_size=18,
                  xaxis_title='sample index',
                  yaxis_title='distances')
fig.show()

<a name="cell-id4"></a>
## **5.** Complete Linkage Clustering


**Bağlantıyı Tamamla (Complete Linkage)**

İki küme arasındaki mesafe, her kümedeki iki nokta arasındaki en uzun mesafedir. Sürecin başlangıcında, her öğe kendi başına bir kümededir. Kümeler daha sonra, tüm öğeler aynı kümede olana kadar sırayla daha büyük kümeler halinde birleştirilir. Yöntem aynı zamanda en uzak komşu kümeleme olarak da bilinir. 

<div>
<img src="https://miro.medium.com/max/875/1*_MtnDQmsLh54vlRdo0BkcA.png" width="270"/>
</div>

${\displaystyle D(X,Y)=\max _{x\in X,y\in Y}d(x,y)}$

In [ ]:
complete = AgglomerativeClustering(linkage='complete', n_clusters=3).fit(points)

In [ ]:
complete.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=object)

In [ ]:
centers = complete.cluster_centers_
clusters = complete.clusters_
visualize_clusters(clusters, centers)

In [ ]:
complete.predict([[2, 3],
                [1, 2],
                [2, -1]])

array([1, 1, 2], dtype=object)

In [ ]:
complete_link = linkage(points, 'complete')

fig = ff.create_dendrogram(complete_link)
fig.update_layout(width=1200, 
                  height=500,
                  title='Hierarchical Clustering Dendrogram',
                  title_font_size=18,
                  xaxis_title='sample index',
                  yaxis_title='distances')
fig.show()

#**Referanslar**

*   [Wikipedia - K-means](https://en.wikipedia.org/wiki/K-means_clustering)
*   [K-means Kümeleme](https://medium.com/@ekrem.hatipoglu/machine-learning-clustering-k%C3%BCmeleme-k-means-algorithm-part-13-be33aeef4fc8)
*   [Kümeleme yöntemleri](https://medium.com/deep-learning-turkiye/k%C3%BCmeleme-teknikleri-ve-pythonda-k-means-62d93c624e5c)
*   [Scikit-Learn](https://scikit-learn.org/stable/)
*   [Introduction to K-means Clustering](https://blog.floydhub.com/introduction-to-k-means-clustering-in-python-with-scikit-learn/)
*   [K-means Algorithm](https://anderfernandez.com/en/blog/kmeans-algorithm-python/)
*   [K-means](https://jakevdp.github.io/PythonDataScienceHandbook/05.11-k-means.html) 
*   [Visualization](https://blog.dominodatalab.com/visualizing-data-with-plotly-and-domino/)
*   [K-means in python-1](https://github.com/siddheshk/Faster-Kmeans/blob/master/Code/kmeans.py)
*   [K-means in python-2](https://github.com/stuntgoat/kmeans)
*   [K-means in python-3](https://github.com/kjahan/k_means)
*   [Clustering for Crypto](https://towardsdatascience.com/clustering-ethereum-addresses-18aeca61919d)
*   [Dendrograms](https://plotly.com/python/dendrogram/)
*   [Single Linkage-Kaggle](https://www.kaggle.com/barelydedicated/hierarchical-clustering-single-link)
*   [Complete Linkage-Kaggle](https://www.kaggle.com/barelydedicated/hierarchical-clustering-complete-link)
*   [Agglomerative Clustering](https://towardsdatascience.com/machine-learning-algorithms-part-12-hierarchical-agglomerative-clustering-example-in-python-1e18e0075019)
*   [Single Linkage](https://en.wikipedia.org/wiki/Single-linkage_clustering)
*   [Complete Linkage](https://en.wikipedia.org/wiki/Complete-linkage_clustering)
*   [Modern hierarchical, agglomerative clustering algorithms](https://arxiv.org/pdf/1109.2378v1.pdf)
*   [Fast optimal leaf ordering for hierarchical clustering](https://doi.org/10.1093/bioinformatics/17.suppl_1.S22)
*   [Single/Complete Clustering - Stanford](https://nlp.stanford.edu/IR-book/html/htmledition/single-link-and-complete-link-clustering-1.html)